In [1]:
import pandas as pd
import numpy as np

In [2]:
train_merged=pd.read_csv('./train_merged.csv')
df=train_merged

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
train_merged=pd.read_csv('./train_merged.csv')
df=train_merged
v = CountVectorizer(lowercase=False,tokenizer=lambda x:x)
cur=df.groupby("user_id")["creative_id"].apply(list)
v_fit = v.fit_transform(cur)
display(v_fit)
sparse.save_npz('./v_fit.npz', v_fit)

<900000x2481135 sparse matrix of type '<class 'numpy.int64'>'
	with 27608868 stored elements in Compressed Sparse Row format>

In [4]:
%reset -f
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
agg=pd.read_csv('agg.csv')
agg.astype(int)
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["gender"].iloc[:num_train]
y_test=user_info["gender"].iloc[num_train:]
X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()

In [8]:
# !pip install lightgbm
import lightgbm as lgb
display(X_train,y_train)
display(X_test,y_test)
train_data = lgb.Dataset(X_train,y_train)
# train_data.save_binary('CountVectorizer_gender.bin')

<720000x2481149 sparse matrix of type '<class 'numpy.float64'>'
	with 32167131 stored elements in Compressed Sparse Row format>

0         0
1         0
2         0
3         0
4         1
         ..
719995    1
719996    1
719997    0
719998    0
719999    1
Name: gender, Length: 720000, dtype: int64

<180000x2481149 sparse matrix of type '<class 'numpy.float64'>'
	with 8041737 stored elements in Compressed Sparse Row format>

720000    0
720001    0
720002    0
720003    0
720004    0
         ..
899995    0
899996    0
899997    0
899998    0
899999    1
Name: gender, Length: 180000, dtype: int64

In [7]:
# %reset -f
import lightgbm as lgb
lgb_params = {'num_leaves': 2**7-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':10,
             }
# with open('lgb.txt', 'a') as f:
#     f.write(str(lgb_params))
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)

start training
[1]	cv_agg's auc: 0.500013 + 0.000564483
[2]	cv_agg's auc: 0.50014 + 0.000603008
[3]	cv_agg's auc: 0.499881 + 0.00082602
[4]	cv_agg's auc: 0.499795 + 0.00106572
[5]	cv_agg's auc: 0.590675 + 0.00161227
[6]	cv_agg's auc: 0.589431 + 0.00167528
[7]	cv_agg's auc: 0.593548 + 0.00184955
[8]	cv_agg's auc: 0.59284 + 0.00175801
[9]	cv_agg's auc: 0.59482 + 0.00177275
[10]	cv_agg's auc: 0.594363 + 0.00182287
[11]	cv_agg's auc: 0.593967 + 0.00177995
[12]	cv_agg's auc: 0.593582 + 0.00176067
[13]	cv_agg's auc: 0.595016 + 0.00201939
[14]	cv_agg's auc: 0.595776 + 0.00215367
[15]	cv_agg's auc: 0.596331 + 0.00212959
[16]	cv_agg's auc: 0.596574 + 0.00213386
[17]	cv_agg's auc: 0.59678 + 0.00213544
[18]	cv_agg's auc: 0.59654 + 0.00215957
[19]	cv_agg's auc: 0.596283 + 0.00215937
[20]	cv_agg's auc: 0.596127 + 0.0021388
[21]	cv_agg's auc: 0.595901 + 0.00205704
[22]	cv_agg's auc: 0.596007 + 0.00197206
[23]	cv_agg's auc: 0.595857 + 0.00198712
[24]	cv_agg's auc: 0.595677 + 0.00200571
[25]	cv_agg's 

KeyboardInterrupt: 

In [ ]:
# %%capture cap --no-stderr
# !pip install lightgbm
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["age","user_id"])
# convert {1,2} to binary value {0,1}
df["gender"]-=1
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("gender")
X_train=train
y_test=test.pop("gender")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)



train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('gender.bin')


In [2]:
# %%capture cap --no-stderr
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_gender.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
              'seed':1024,
              'nthread':10,
             }
# with open('lgb.txt', 'a') as f:
#     f.write(str(lgb_params))
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)
# print('best num_boost_round:', len(cv_results['auc-mean']))
# print('best cv score:', cv_results['auc-mean'])

KeyboardInterrupt: 

In [12]:
# with open('lgb.txt', 'w') as f:
#     f.write(cap.stdout)

NameError: name 'cap' is not defined

In [4]:
# !pip install lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["gender","user_id"])
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("age")
X_train=train
y_test=test.pop("age")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)
train_data.save_binary('age.bin')

NameError: name 'train_merged' is not defined

In [ ]:
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('age.bin')
params = {'num_leaves': 2**7-1,
          'min_data_in_leaf': 25, 
          'objective':'regression_l2',
          'max_depth': -1,
          'learning_rate': 0.1,
          'min_child_samples': 20,
          'boosting': 'gbdt',
          'feature_fraction': 0.6,
          'bagging_fraction': 0.9,
          'bagging_seed': 11,
          'metric': 'mae',
          'seed':1024,
          'lambda_l1': 0.2,
          'nthread':12,
         }
print("start training")
cv_results = lgb.cv(
    params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=10, show_stdv=True, seed=0)
print('best num_boost_round:', len(cv_results['mae-mean']))
print('best cv score:', cv_results['mae-mean'])
# bst = lgb.train(param, train_data, num_round,categorical_feature=features)
# bst.save_model('test_model.txt')

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'nthread':12,
          'learning_rate':0.1
          }
 
### 交叉验证(调参)
print('交叉验证')
max_auc = float('0')
best_params = {}
 
# 准确率
print("调参1：提高准确率")
for num_leaves in range(5,100,5):
    for max_depth in range(3,8,1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
 
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
            
        if mean_auc >= max_auc:
            max_auc = mean_auc
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
if 'num_leaves' and 'max_depth' in best_params.keys():          
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']
 
# 过拟合
print("调参2：降低过拟合")
for max_bin in range(5,256,10):
    for min_data_in_leaf in range(1,102,10):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc = mean_auc
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']
    params['max_bin'] = best_params['max_bin']
 
print("调参3：降低过拟合")
for feature_fraction in [0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc=mean_auc
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq
 
if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']
 
 
print("调参4：降低过拟合")
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
                
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
        if mean_auc >= max_auc:
            max_auc=mean_auc
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']
 
print("调参5：降低过拟合2")
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                        params,
                        lgb_train,
                        seed=1,
                        nfold=5,
                        metrics=['auc'],
                        early_stopping_rounds=10,
                        verbose_eval=True
                        )
            
    mean_auc = pd.Series(cv_results['auc-mean']).max()
    boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
    if mean_auc >= max_auc:
        max_auc=mean_auc
        
        best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
 
print(best_params)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])